# Objective – Phase 1	 

In [2]:
import pandas as pd
import statsmodels.api as sm

## Data Considerations
Based on your first report, the Bank has strategically binned each of the continuous variables in 
the data set to help facilitate any further analysis.  
- For any variable with missing values, change the data to include a missing category 
instead of a missing value for the categorical variable.  
    - (HINT: Now all variables should be categorized (treated as categorical variables 
so no more continuous variable assumptions) and without missing values. Banks 
do this for more advanced modeling purposes that we will talk about in the 
spring.) 
- Check each variable for separation concerns. Document in the report and adjust any 
variables with complete or quasi-separation concerns. 

### Handling Missing Values

In [3]:
df = pd.read_csv("insurance_t_bin.csv")
print(df.shape)
df.head()

(8495, 48)


,DDA,CASHBK,DIRDEP,NSF,SAV,ATM,CD,IRA,LOC,INV,...,INVBAL_BIN,ILSBAL_BIN,MMBAL_BIN,MTGBAL_BIN,CCBAL_BIN,INCOME_BIN,LORES_BIN,HMVAL_BIN,AGE_BIN,CRSCORE_BIN
0,1,0,1,0,0,1,0,0,0,0.0,...,01 <= 1025,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,01 <= 0.9,01 <= 50,02 > 5,01 <= 98,02 > 50,01 <= 675
1,0,0,0,0,0,0,0,0,0,0.0,...,01 <= 1025,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,01 <= 0.9,01 <= 50,02 > 5,01 <= 98,02 > 50,01 <= 675
2,1,0,1,0,0,0,0,0,0,0.0,...,01 <= 1025,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,02 > 0.9,01 <= 50,01 <= 5,02 <= 122,02 > 50,01 <= 675
3,1,0,0,1,1,1,0,0,0,NaN,...,00 Miss,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,00 Miss,01 <= 50,01 <= 5,02 <= 122,01 <= 50,01 <= 675
4,1,0,0,1,1,1,1,0,0,NaN,...,00 Miss,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,00 Miss,01 <= 50,02 > 5,01 <= 98,02 > 50,02 > 675


In [4]:
df.isnull().sum()

DDA               0
CASHBK            0
DIRDEP            0
NSF               0
SAV               0
ATM               0
CD                0
IRA               0
LOC               0
INV            1075
ILS               0
MM                0
MMCRED            0
MTG               0
CC             1075
CCPURC         1075
SDB               0
HMOWN          1463
MOVED             0
INAREA            0
INS               0
BRANCH            0
RES               0
ACCTAGE_BIN       0
DDABAL_BIN        0
DEP_BIN           0
DEPAMT_BIN        0
CHECKS_BIN        0
NSFAMT_BIN        0
PHONE_BIN         0
TELLER_BIN        0
SAVBAL_BIN        0
ATMAMT_BIN        0
POS_BIN           0
POSAMT_BIN        0
CDBAL_BIN         0
IRABAL_BIN        0
LOCBAL_BIN        0
INVBAL_BIN        0
ILSBAL_BIN        0
MMBAL_BIN         0
MTGBAL_BIN        0
CCBAL_BIN         0
INCOME_BIN        0
LORES_BIN         0
HMVAL_BIN         0
AGE_BIN           0
CRSCORE_BIN       0
dtype: int64

Looks like INV, CC, CCPURC, HMOWN all have missing data

In [5]:
miss = ['INV', 'CC', 'CCPURC', 'HMOWN']
for x in miss:
    print(f"Unique values in {x}: {df[x].unique()}")

Unique values in INV: [ 0. nan  1.]
Unique values in CC: [ 1. nan  0.]
Unique values in CCPURC: [ 1.  0. nan  2.  3.  4.]
Unique values in HMOWN: [ 1.  0. nan]


In [6]:
# replacing missing values with 'Missing'
for col in miss:
    df[col] = df[col].fillna('Missing')
    df[col] = df[col].astype('category')

In [14]:
# correctly encode the data in preperation on the logistic regression
binary_columns = [col for col in df.columns if df[col].nunique() == 2]
non_binary_columns = [col for col in df.columns if col not in binary_columns]

df_encoded = pd.get_dummies(df, columns=non_binary_columns, drop_first=True, dtype=int)

pd.set_option('display.max_columns', 500)
df_encoded.head()

,DDA,DIRDEP,NSF,SAV,ATM,CD,IRA,LOC,ILS,MM,MTG,SDB,MOVED,INAREA,INS,NSFAMT_BIN,IRABAL_BIN,LOCBAL_BIN,ILSBAL_BIN,MMBAL_BIN,MTGBAL_BIN,CASHBK_1,CASHBK_2,INV_1.0,INV_Missing,MMCRED_1,MMCRED_2,MMCRED_3,MMCRED_5,CC_1.0,CC_Missing,CCPURC_1.0,CCPURC_2.0,CCPURC_3.0,CCPURC_4.0,CCPURC_Missing,HMOWN_1.0,HMOWN_Missing,BRANCH_B10,BRANCH_B11,BRANCH_B12,BRANCH_B13,BRANCH_B14,BRANCH_B15,BRANCH_B16,BRANCH_B17,BRANCH_B18,BRANCH_B19,BRANCH_B2,BRANCH_B3,BRANCH_B4,BRANCH_B5,BRANCH_B6,BRANCH_B7,BRANCH_B8,BRANCH_B9,RES_S,RES_U,ACCTAGE_BIN_01 <= 19.6,ACCTAGE_BIN_02 > 19.6,DDABAL_BIN_02 <= 82.57,DDABAL_BIN_03 <= 304.95,DDABAL_BIN_04 <= 764.81,DDABAL_BIN_05 <= 1248.47,DDABAL_BIN_06 <= 2188.02,DDABAL_BIN_07 <= 6126.24,DDABAL_BIN_08 > 6126.24,DEP_BIN_02 <= 3,DEP_BIN_03 > 3,DEPAMT_BIN_02 <= 686.49,DEPAMT_BIN_03 <= 2190.32,DEPAMT_BIN_04 <= 6425.57,DEPAMT_BIN_05 > 6425.57,CHECKS_BIN_02 <= 2,CHECKS_BIN_03 <= 4,CHECKS_BIN_04 > 4,PHONE_BIN_01 <= 0,PHONE_BIN_02 <= 1,PHONE_BIN_03 > 1,TELLER_BIN_02 <= 3,TELLER_BIN_03 > 3,SAVBAL_BIN_02 <= 61.25,SAVBAL_BIN_03 <= 265.87,SAVBAL_BIN_04 <= 1259.45,SAVBAL_BIN_05 <= 2962.02,SAVBAL_BIN_06 <= 8334.97,SAVBAL_BIN_07 > 8334.97,ATMAMT_BIN_02 <= 3688.63,ATMAMT_BIN_03 > 3688.63,POS_BIN_01 <= 0,POS_BIN_02 > 0,POSAMT_BIN_01 <= 250,POSAMT_BIN_02 > 250,CDBAL_BIN_02 <= 9200,CDBAL_BIN_03 > 9200,INVBAL_BIN_01 <= 1025,INVBAL_BIN_02 > 1025,CCBAL_BIN_01 <= 0.9,CCBAL_BIN_02 > 0.9,INCOME_BIN_01 <= 50,INCOME_BIN_02 > 50,LORES_BIN_01 <= 5,LORES_BIN_02 > 5,HMVAL_BIN_01 <= 98,HMVAL_BIN_02 <= 122,HMVAL_BIN_03 <= 142,HMVAL_BIN_04 > 142,AGE_BIN_01 <= 50,AGE_BIN_02 > 50,CRSCORE_BIN_01 <= 675,CRSCORE_BIN_02 > 675
0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,01 <= 6.65,01 <= 0.1,01 <= 5000,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1,1,0,0,0,0,1,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,01 <= 6.65,01 <= 0.1,01 <= 5000,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,0,1,1,0,0,0,0,1,1,0
2,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,01 <= 6.65,01 <= 0.1,01 <= 5000,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,1,0,1,0,0,1,0,0,0,1,1,0
3,1,0,1,1,1,0,0,0,0,0,0,1,0,1,0,01 <= 6.65,01 <= 0.1,01 <= 5000,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0
4,1,0,1,1,1,1,0,0,0,0,0,0,0,1,0,02 > 6.65,01 <= 0.1,01 <= 5000,01 <= 9690.09,01 <= 1031.1401,01 <= 100000,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,1


### Checking for Linear Separation/Convergence Problems

apperently statsmodels and scikit-learn will give warnings and will provide some summary stats after model is fitted. Thus, I will skip this part and check after the model is fitted.

## Model Building
Build a main effects only binary logistic regression model to predict the purchase of the 
insurance product. 
- Use backward selection to do the variable selection – the Bank currently uses 𝛼 = 0.002 
and p-values to perform backward, but is open to another technique and/or significance 
level if documented in your report.  
- Report the final variables from this model ranked by p-value.  
    - (HINT: Even if you choose to not use p-values to select your variables, you 
should still rank all final variables by their p-value in this report.)

In [15]:
y = df_encoded.INS
X = df_encoded.drop('INS', axis=1)
print(X.shape)
print(y.shape)

(8495, 110)
(8495,)


In [16]:
import numpy as np
X = np.asarray(X)
y = np.asarray(y)

In [17]:
def backward_selection(X, y, significance_level=0.02):
    """
    Perform backward selection to remove variables with p-values above the significance level.

    Parameters:
    X: DataFrame of independent variables (predictors)
    y: Series of dependent variable (target)
    significance_level: P-value threshold for retaining variables (default 0.05)

    Returns:
    A list of variables that remain in the model after backward selection.
    """
    # Adding a constant to the model (intercept)
    X = sm.add_constant(X)
    
    # Fit the model with all variables
    model = sm.Logit(y, X).fit(disp=False)  # 'disp=False' suppresses output during each iteration
    
    # Start backward elimination
    while True:
        # Get p-values from the current model
        p_values = model.pvalues
        
        # Get the predictor with the highest p-value
        max_p_value = p_values.max()
        
        # Check if the highest p-value exceeds the significance level
        if max_p_value > significance_level:
            # Remove the predictor with the highest p-value
            max_p_var = p_values.idxmax()  # Variable with max p-value
            X = X.drop(columns=[max_p_var])
            
            # Refit the model without this variable
            model = sm.Logit(y, X).fit(disp=False)
        else:
            break
    
    # Return the remaining predictors
    return X.columns.tolist()

selected_variables = backward_selection(X, y, significance_level=0.02)
print("Selected variables after backward selection:", selected_variables)



TypeError: unsupported operand type(s) for -: 'str' and 'str'